## Quickstart: $J/\Psi \rightarrow \gamma \pi^0 \pi^0$ decay

In this quickstart example it is shown how to use ComPWA via the python interface. The workflow is:

1. Create a model for the decay.
2. Generate a Monte Carlo data sample (hit & miss) using this model. 
3. Perform a fit on the data sample using the Minuit2 interface.
4. Visualize the data and the fit result!

Let's go!

First we `import` the necessary expert system module parts

In [1]:
from pycompwa.expertsystem.ui.system_control import (
    StateTransitionManager, InteractionTypes)
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityDecayAmplitudeGeneratorXML)
from pycompwa.expertsystem.topology.graph import (
    get_intermediate_state_edges)

# just a little function to print the intermediate states
def print_intermediate_states(solutions):
    print("intermediate states:")
    intermediate_states = set()
    for g in solutions:
        edge_id = get_intermediate_state_edges(g)[0]
        intermediate_states.add(g.edge_props[edge_id]['@Name'])
    print(intermediate_states)

### Step 1: Creating the Decay Model

#### 1.1. Define problem set

First we define the boundary conditions of our physics problem, such as

- initial state
- final state
- formalism type
- ...

Pass all of that information to the `StateTransitionManager`, which is the main user interface class of the ComPWA expert system

In [ ]:
initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma", [-1, 1]), ("pi0", [0]), ("pi0", [0])]

tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

#### 1.2. Preparation
Create all topology graphs using the isobar model (two-body decays).

Also initialize the graphs with the initial and final state. Remember that each interaction node defines their own set of conservation laws. The `StateTransitionManager` (STM) defines three interaction types:

| Interaction | Strength |
| --- | --- |
| strong | $60$ |
| electromagnetic (EM) | $1$ |
| weak | $10^{-4}$ |

Be default all three are used in the preparation stage. `prepare_graphs()` of the STM generates graphs with all possible combinations of interaction nodes. An overall interaction strength is assigned to each graph, and they are grouped according to this strength.

In [ ]:
graph_interaction_settings_groups = tbd_manager.prepare_graphs()

#### 1.3. Finding solutions
If you are happy with the automatic settings generated by the StateTransitionManager, just go directly to the solving!

In [ ]:
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Ok, now we have a lot of solutions that are actually heavily supressed (involve two weak decays). In general you can modify the dictionary return by `prepare_graphs()` directly.

The STM also comes with a functionality to globally choose the allowed interaction types (`set_allowed_interaction_types()`). Go ahead and **disable** the **EM** and **weak** interaction!

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)
print("found " + str(len(solutions)) + " solutions!")

Huh, what happened here? Actually, since a **gamma particle** appears, the expert system knows that there must be **at least one EM interaction** involved. As a consequence no graphs are prepared for this setting!

In [ ]:
print(graph_interaction_settings_groups)

So let's include the EM interaction...

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Great! Now we selected the solutions that are contribution the strongest. However, be aware that there are more effects that can suppress certain decays. For example branching ratios. In this example $J/\Psi$ can decay into $\pi^0 + \rho^0$ or $\pi^0 + \omega$.

| decay | branching ratio |
| --- | --- |
| $$\omega \rightarrow \gamma+\pi^0$$ | 0.0828 |
| $$\rho^0 \rightarrow \gamma+\pi^0$$ | 0.0006 |

Unfortunately the $\rho^0$ decays mainly into $\pi+\pi$, not $\gamma+\pi^0$. Hence it is suppressed. This information is currently not known to the expert system.
But you can also tell the expert system, which particles you want to allow as intermediate states.

In [ ]:
# particles are found by name comparison; so i.e. f2 will find all f2's and f all f's
tbd_manager.allowed_intermediate_particles = ['f']
#tbd_manager.allowed_intermediate_particles = ['f2, f0']

(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Now we have selected all amplitudes that involve **f** states. At this point we are all set to generate some data using this amplitude model!

In [ ]:
xml_generator = HelicityDecayAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

### Step 2: Creating a data sample

In this section we will use the amplitude model created above to generate a data sample via hit & miss Monte Carlo.

Using this amplitude model in the c++ side of ComPWA is simple. The `create_intensity_and_kinematics()` helper function builds an Intensity and Kinematics object, as specified in the xml file. These can be used to generate our data sample.

In [ ]:
# pycompwa is the python interface to ComPWA's c++ modules
import pycompwa as pwa

# create the intensity and kinematics
intensity, kinematics = pwa.create_intensity_and_kinematics('model.xml')

# use the RootGenerator to generate N particle events, as specified in the kinematics info
generator = pwa.RootGenerator(kinematics.get_particle_state_transition_kinematics_info(), 12345)

# generate data sample
datasample = pwa.generate(5000, kinematics, generator, intensity)

### Step 3: Fitting

All parameter defined and used by the **Intensity** object, can be obtain for it by using the `parameters()` function. Just pass it an empty `ParameterList` object.

In [ ]:
par_list = pwa.ParameterList()
intensity.add_unique_parameters_to(par_list)
fit_parameters = par_list.get_fit_parameters()

Let's save the true parameters in a dictionary so we can compare the fitted values later on. Notice that the `get_fit_parameters()` returns a special object that behave similar to a python list. The contents of the list are FitParameter objects, that have attributes `name, value, error, is_fixed`. The name and error attributes are read only.

In [ ]:
true_parameters = {x.name: x.value for x in fit_parameters if not x.is_fixed}
print("number of free fit parameters:", len(true_parameters))
print(true_parameters)

To make the fit a bit more interesting, we modify one of the parameters to a different initial value then the true value.

In [ ]:
idx = fit_parameters.index("Magnitude_J/psi_to_f2(1270)_0+gamma_-1;f2(1270)_to_pi0_0;")
print("before:", fit_parameters[idx])
fit_parameters[idx].value = 2.0
print("after:",fit_parameters[idx])
# we can also fix or free parameters here
fit_parameters[fit_parameters.index(
    'Phase_J/psi_to_f2(1270)_0+gamma_-1;f2(1270)_to_pi0_0;')].is_fixed = True
print("should be fixed now.... \n",fit_parameters[fit_parameters.index(
    'Phase_J/psi_to_f2(1270)_0+gamma_-1;f2(1270)_to_pi0_0;')])

We want to use an unbinned minimum log likelihood estimator to find the set of fit parameters of the intensity, that describe our data best. However in this example the intensity is not normalized. The minimum log likelihood estimator can handle the normalization for us, but a phase space data sample has to be supplied.

**Note**: The intensities are evaluated with data points or parameterlists and not events, so they have to be converted using the kinematics class first. This way the data calculation is automatically cached!

In [ ]:
phspsample = pwa.generate_phsp(100000, generator)
datasample.convert_events_to_parameterlist(kinematics)
phspsample.convert_events_to_parameterlist(kinematics)

Now it's time to start up a set up a fit, which is quite simple.
1. First create an estimator instance of your choice, here a unbinned minimum log likelihood (`MinLogLH`). Notice that we use the function tree feature. This creates a full evaluation tree of the intensity, converting the data into a vertical layout (for cache optimization) and caching intermediate values of the intensity. It can greatly enhance the fit performance (here: 7 sec -> 150ms)! We use the `create_unbinned_log_likelihood_function_tree_estimator()` helper function to create the function tree version of our log LH estimator.
2. Then create an optimizer instance of your choice, here Minuit2 (`MinuitIF`).

**Note**: The runtime of the fit (with 15 free parameters) can take a couple of minutes (~5 min)

In [ ]:
#the next line creates a normal log likelihood estimator (without the function tree feature)
#esti = pwa.MinLogLH(intensity, datasample, phspsample)

estimator = pwa.create_unbinned_log_likelihood_function_tree_estimator(intensity, datasample, phspsample)
minuitif = pwa.MinuitIF(estimator, par_list)
minuitif.enable_hesse(True)

result = minuitif.minimize(par_list)

Let's check if the fit parameters are "close to" the true values

In [ ]:
fitresult_parameters = {x.name: (x.value, x.error) for x in fit_parameters if not x.is_fixed}
for key, value in fitresult_parameters.items():
    print(key, " fit result:", "{0:.3f}".format(value[0]), "+-", 
          "({0:.3f},".format(value[1][0]), "{0:.3f})".format(value[1][1]),
          " true:", "{0:.3f}".format(true_parameters[key])
         )

### Step 4: Visualization
Let's go ahead and make a Dalitz plot of the data sample and our fit result. ComPWA ships with a little plotting module to help you generate some common plots using matplotlib. Before we can utilize this, the data has to be extended.

Since our model only includes one specific decay topology the `HelicityKinematics` class only generates the kinematic variables needed to evaluate the Intensity. In order to make a Dalitz plot, also the kinematic variables from the other subsystems are needed. They can be created with `create_all_subsystems()`. However the conversion from the event samples have to be performed again!

In [ ]:
kinematics.create_all_subsystems()
datasample.convert_events_to_datapoints(kinematics)
phspsample.convert_events_to_datapoints(kinematics)

This step is optional: create a ROOT file containing all of the information inside a TTree and read it back in again

In [ ]:
pwa.create_rootplotdata("rootplot.root", kinematics, data_sample=datasample, 
                        phsp_sample=phspsample, intensity=intensity)

from Plotting.ROOT.rootplotdatareader import open_compwa_plot_data

plot_data = open_compwa_plot_data("rootplot.root")

The ComPWA python interface has two helper functions `create_data_array()` `create_fitresult_array()` that allow the datapoins to be exported as two-dimensional lists. These can be used to create a numpy record arrays. Then the **Plotting** module is able to create common plots like the Dalitz plot.

In [ ]:
# Plotting
from Plotting.plot import (
    make_dalitz_plots, PlotData, create_nprecord,
    make_difference_plot_2d
)

# use the direct data point access
var_names, dataarray = pwa.create_data_array(datasample)
data_record = create_nprecord(var_names, dataarray)

fitres_var_names, fitres_dataarray = pwa.create_fitresult_array(intensity, phspsample)
fitresult_record = create_nprecord(fitres_var_names, fitres_dataarray)

plot_data = PlotData(data_record=data_record, fit_result_data_record=fitresult_record)
#plot_data.particle_id_to_name_mapping = data_points.get_finalstate_id_to_name_mapping()
# plot a 2d histogram
make_dalitz_plots(plot_data, ["mSq_3_4_vs_2", "mSq_2_4_vs_3"], bins=50)

To make the differences more clearly visible we can create a 2d difference plot

In [ ]:
make_difference_plot_2d(plot_data, ["mSq_3_4_vs_2", "mSq_2_4_vs_3"], bins=50)

That's it. You can check some of the other examples to learn about more detailed features of ComPWA.

And give us feedback or contribute ;)!